<a href="https://colab.research.google.com/github/marmik10/Movie-Suggestion-System/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import difflib # when we get user input, there are chances there might be incorrect spelling,
               # hence it used to find the closest data available from dataset
from sklearn.feature_extraction.text import TfidfVectorizer #converting the textual into numerical value
from sklearn.metrics.pairwise import cosine_similarity

Data Pre - Processing

In [2]:
movies_data = pd.read_csv('/content/movies.csv')

In [3]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies_data.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [5]:
# number of rows and coloumns in the dataset

movies_data.shape

(4803, 24)

In [6]:
# Selecting the relevant features for recommendation

selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [7]:
# combining all features

combined_features = movies_data['genres'] + movies_data['keywords'] + movies_data['tagline'] + movies_data['cast'] + movies_data['director']
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799                                                  NaN
4800                                                  NaN
4801                                                  NaN
4802                                                  NaN
Length: 4803, dtype: object


In [8]:
# converting the text data into vector values or numerical values

vectorizer = TfidfVectorizer()

In [11]:
feature_vectors = vectorizer.fit_transform(combined_features.apply(lambda x: np.str_(x)))

In [12]:
print (feature_vectors)

  (0, 3037)	0.16900102823868982
  (0, 17413)	0.2695729644606029
  (0, 13920)	0.15875789964000211
  (0, 12162)	0.23112705876096243
  (0, 19563)	0.16273756817398058
  (0, 22364)	0.19331393611073014
  (0, 18867)	0.19890918242291228
  (0, 18026)	0.21458585411113668
  (0, 23204)	0.2021425797624835
  (0, 22922)	0.23112705876096243
  (0, 18052)	0.15753831304129623
  (0, 15553)	0.2571296901119497
  (0, 15215)	0.08851165996992062
  (0, 22902)	0.1270417602096386
  (0, 20359)	0.07595648434647574
  (0, 19141)	0.2695729644606029
  (0, 3976)	0.24830104646019635
  (0, 22178)	0.13433563953504693
  (0, 19257)	0.3391711924209033
  (0, 8503)	0.16468672413442242
  (0, 3785)	0.2201811004233188
  (0, 7779)	0.2695729644606029
  (0, 18320)	0.10189402788810341
  (0, 7458)	0.11926369461762627
  (0, 381)	0.09317937865464082
  :	:
  (4798, 3177)	0.15280520679880832
  (4798, 9312)	0.17760500872466448
  (4798, 21208)	0.31961313478980063
  (4798, 13367)	0.17000956772347908
  (4798, 11081)	0.15980656739490032
  (4798

In [13]:
# getting the similarity scores from cosine similarity

similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.07055384 0.01602002 ... 0.         0.         0.        ]
 [0.07055384 1.         0.02878279 ... 0.         0.         0.        ]
 [0.01602002 0.02878279 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         1.         1.        ]
 [0.         0.         0.         ... 1.         1.         1.        ]
 [0.         0.         0.         ... 1.         1.         1.        ]]


In [14]:
print(similarity.shape)

(4803, 4803)


In [15]:
# here first one presents all the movie indexes and other one shows similarity scores

In [16]:
# getting a movie name from the user

movie_name = input('Enter the movie name: ')

Enter the movie name: Batman Begins


In [19]:
# creating a list of all movie present in the dataset

list_of_item = movies_data['title'].tolist()

In [20]:
print(list_of_item)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [24]:
# finding the close match for movie name

find_close_match = difflib.get_close_matches(movie_name, list_of_item)
print(find_close_match)

['Batman Begins', 'Batman Returns', 'Top Cat Begins']


In [25]:
close_match = find_close_match[0]
print(close_match)

Batman Begins


In [27]:
# find the index of the movie with the title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

119


In [28]:
# getting a similar movie

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.012933261609707848), (1, 0.015624972385912097), (2, 0.02992374348684267), (3, 0.3580948588767827), (4, 0.0068028644309791496), (5, 0.007217343514334232), (6, 0.0), (7, 0.007040639168707792), (8, 0.013787438369232692), (9, 0.04738309015287792), (10, 0.0), (11, 0.04688049817944867), (12, 0.0), (13, 0.012571059197988267), (14, 0.11311178184619239), (15, 0.0), (16, 0.006720104638564932), (17, 0.00689354658249749), (18, 0.025298619340654586), (19, 0.01669409929813603), (20, 0.006800670165927522), (21, 0.0456130344062346), (22, 0.0057493532541158625), (23, 0.010350312076809939), (24, 0.005601674043364223), (25, 0.0), (26, 0.0074809282397581885), (27, 0.006331318791854319), (28, 0.006792423729165181), (29, 0.030645713821213373), (30, 0.0070123481048407444), (31, 0.018905133702665014), (32, 0.0), (33, 0.007118753967240736), (34, 0.0), (35, 0.007517482674775767), (36, 0.006900624101156016), (37, 0.0), (38, 0.007136452006798777), (39, 0.028950712552527572), (40, 0.065066088236708), (41, 0

In [29]:
len(similarity_score)

4803

In [30]:
# sorting the movies on based of the similarity score

sorted_similar_movie = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movie)

[(119, 1.0000000000000002), (3, 0.3580948588767827), (65, 0.34039858298309605), (1196, 0.21389313189671835), (210, 0.17294528761278258), (428, 0.16077164267071245), (813, 0.15540742215763712), (1359, 0.13055453210275947), (299, 0.12107836700547339), (3332, 0.12029769266299381), (1815, 0.11470104086604856), (14, 0.11311178184619239), (1035, 0.10903549505140424), (252, 0.1087709978024291), (873, 0.1075968868130611), (695, 0.10673641788768064), (1427, 0.1058781250765618), (95, 0.10271890223174461), (3463, 0.10093048406776554), (303, 0.10042823756500828), (4610, 0.0973771516215747), (499, 0.0970716702163188), (870, 0.09589184615194053), (2790, 0.09516421301231065), (280, 0.09260850581244938), (2942, 0.0909844980702321), (1033, 0.09076764547485087), (72, 0.09034322226847105), (3425, 0.08925189799012408), (96, 0.08911058830060375), (1924, 0.08900899721088731), (3395, 0.08895517370632092), (1041, 0.08893080435619118), (3309, 0.08759013267061319), (828, 0.0855699935756026), (535, 0.08510757638

In [34]:
# print the similar movie based on index that we got

print('Movies suggested for you is: \n')

i = 1
for movie in sorted_similar_movie:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if(i<30):
    print(i, ' ', title_from_index)
    i+=1

Movies suggested for you is: 

1   Batman Begins
2   The Dark Knight Rises
3   The Dark Knight
4   The Prestige
5   Batman & Robin
6   Batman Returns
7   Superman
8   Batman
9   Batman Forever
10   Harry Brown
11   Taken
12   Man of Steel
13   Jonah Hex
14   The Green Hornet
15   Shaft
16   GoldenEye
17   Wonder Boys
18   Interstellar
19   Mr. Nice Guy
20   Catwoman
21   Pieces of April
22   Jack and Jill
23   Superman II
24   True Romance
25   Public Enemies
26   Woman in Gold
27   Insomnia
28   Suicide Squad
29   Go
